In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy import interpolate
%matplotlib inline
sns.set_context("paper")  
sns.set(rc={'figure.figsize': (10, 8)})  # 设置画板大小
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [2]:
DIY_path = './data/DIY_df.csv'
DIY = pd.read_csv(DIY_path, encoding='utf-8')
def split(descp, col_name):
    # split the column into 4 parts according to the quartile
    # return the 3 split point
    tgt = descp[col_name]
    split1 = tgt['25%']
    split2 = tgt['50%']
    split3 = tgt['75%']
    split_points = [split1, split2, split3]
    return split_points
def hierarchy(df):
	col_names = df.columns
	descp = df.describe()
	for col_name in col_names:
		split_points = split(descp, col_name)
		for i in range(len(df[col_name])):
			if df[col_name][i] < split_points[0]:
				df[col_name][i] = 1
			elif df[col_name][i] < split_points[1]:
				df[col_name][i] = 2
			elif df[col_name][i] < split_points[2]:
				df[col_name][i] = 3
			else:
				df[col_name][i] = 4
	return df
hier_DIY = hierarchy(DIY)
# save the dier_DIY dataframe
hier_DIY.to_csv('./data/hier_DIY.csv', encoding='utf-8', index=False)

In [59]:
hier_DIY_path = './data/hier_DIY.csv'

In [60]:
hier_DIY = pd.read_csv(hier_DIY_path, encoding='utf-8')

In [61]:
cols = hier_DIY.columns
for col in cols:
    print(hier_DIY[col].value_counts())

2.0    13268
3.0    10132
4.0     8730
Name: PRNT_EDU, dtype: int64
4.0    11938
3.0    11081
2.0     9111
Name: STDY_EVNRNMNT, dtype: int64
2.0    15910
4.0     8460
3.0     7760
Name: LF_QLTY, dtype: int64
2.0    14584
4.0    11336
3.0     6210
Name: PRNT_CR, dtype: int64
3.0    22975
4.0     9155
Name: EX_RPT, dtype: int64
2.0    15805
4.0     8210
3.0     8115
Name: STY, dtype: int64
3.0    11538
2.0    11359
4.0     9233
Name: PEER, dtype: int64
2.0    12943
3.0    10442
4.0     8745
Name: TCHR, dtype: int64
2.0    12622
4.0    10353
3.0     9155
Name: SKIP, dtype: int64
2.0    12067
3.0    10155
4.0     9908
Name: CLS1, dtype: int64
2.0    15852
4.0     8238
3.0     8040
Name: CLS2, dtype: int64
2.0    15016
4.0     8814
3.0     8300
Name: CLS3, dtype: int64
1.0    8033
4.0    8033
2.0    8032
3.0    8032
Name: KNLDG, dtype: int64
3.0    17660
4.0    14470
Name: REPEAT, dtype: int64


In [ ]:
def Naive_Bayes(df):
	# calculate the prior probability
	prior = df.groupby('class').size()/len(df)
	# calculate the conditional probability
	cond_prob = df.groupby('class').mean()
	# calculate the posterior probability
	posterior = pd.DataFrame(index=prior.index)
	for col in df.columns:
		posterior[col] = df[col].apply(lambda x: cond_prob[col][x])
	posterior = posterior.apply(lambda x: x*prior)
	posterior = posterior.apply(lambda x: x/posterior.sum(axis=1))
	return posterior

In [65]:
prior = df.groupby('PRNT_EDU').size()/len(df)

In [66]:
prior

PRNT_EDU
2.0    0.412947
3.0    0.315344
4.0    0.271709
dtype: float64

In [68]:
cond_prob = df.groupby('PRNT_EDU').mean()
cond_prob

,STDY_EVNRNMNT,LF_QLTY,PRNT_CR,EX_RPT,STY,PEER,TCHR,SKIP,CLS1,CLS2,CLS3,KNLDG,REPEAT
PRNT_EDU,,,,,,,,,,,,,
2.0,3.022762,2.782710,2.946488,3.248267,2.739071,2.932771,2.885891,2.934806,2.907220,2.743141,2.796955,2.562707,3.421691
3.0,3.132846,2.758389,2.861725,3.326984,2.789479,2.917687,2.864094,2.936044,2.948381,2.760462,2.814647,2.417588,3.480458
4.0,3.135052,2.757274,2.869759,3.291867,2.770905,2.954181,2.850286,2.913402,2.953608,2.796220,2.813288,2.500344,3.458992
